In [1]:
import os
import json
from web3 import Web3, HTTPProvider

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

from utils.eth_blocks_utils import get_timestamp_for_block

In [2]:
ALCHEMY_API_KEY = os.environ['ALCHEMY_API_KEY']
web3 = Web3(HTTPProvider(f"https://eth-mainnet.alchemyapi.io/v2/{ALCHEMY_API_KEY}"))

In [3]:
pool_addr = "0x0f9cb53Ebe405d49A0bbdBD291A65Ff571bC83e1"

In [4]:
with open("./usdn.json", 'r') as f:
    abi = json.load(f)
pool = web3.eth.contract(address=pool_addr, abi=abi)
pool

In [5]:
current_block = web3.eth.get_block_number()
current_block

13679960

In [6]:
genesis_block = 11010514

In [7]:
pool_remove_liquidity_filter = pool.events.RemoveLiquidity.createFilter(
    fromBlock=genesis_block,
    toBlock=current_block
)
pool_remove_liquidity_one_filter = pool.events.RemoveLiquidityOne.createFilter(
    fromBlock=genesis_block,
    toBlock=current_block
)
pool_remove_liquidity_imbalance_filter = pool.events.RemoveLiquidityImbalance.createFilter(
    fromBlock=genesis_block,
    toBlock=current_block
)

In [11]:
block_steps = list(range(genesis_block, current_block, int(0.25 * round(current_block - genesis_block))))
block_ranges = list(zip(block_steps, block_steps[1:]))
block_ranges

[(11010514, 11677875),
 (11677875, 12345236),
 (12345236, 13012597),
 (13012597, 13679958)]

In [25]:
liq_providers = {
    "addr": [],
    "usdn": [],
    "3crv": [],
    "block": []
}

for (block_start, block_end) in block_ranges:

    pool_add_liquidity_filter = pool.events.AddLiquidity.createFilter(
        fromBlock=block_start,
        toBlock=block_end
    )

    add_liquidity_events = pool_add_liquidity_filter.get_all_entries()

    liq_providers['addr'].extend([event['args']['provider'] for event in add_liquidity_events])
    liq_providers['usdn'].extend([event['args']['token_amounts'][0] * 1e-18 for event in add_liquidity_events])
    liq_providers['3crv'].extend([event['args']['token_amounts'][1] * 1e-18 for event in add_liquidity_events])
    liq_providers['block'].extend([event['blockNumber'] for event in add_liquidity_events])


In [26]:
df_liq_provider_events = pd.DataFrame(liq_providers)
df_liq_provider_events

,addr,usdn,3crv,block
0,0x35796DAc54f144DFBAD1441Ec7C32313A7c29F39,3.00,2.993241,11010708
1,0x35796DAc54f144DFBAD1441Ec7C32313A7c29F39,0.00,0.000002,11010724
2,0x35796DAc54f144DFBAD1441Ec7C32313A7c29F39,50000.00,49712.700063,11010954
3,0xF7AE063de5EF4BCBb98B2Bd27afb66C59D885Fc0,0.00,2000.000000,11011537
4,0x35796DAc54f144DFBAD1441Ec7C32313A7c29F39,1005.63,0.000000,11011601
...,...,...,...,...
12533,0x094d12e5b541784701FD8d65F11fc0598FBC6332,0.00,7775.200614,13667912
12534,0x094d12e5b541784701FD8d65F11fc0598FBC6332,0.00,71274.775219,13668743
12535,0x094d12e5b541784701FD8d65F11fc0598FBC6332,0.00,551.484707,13675015
12536,0x094d12e5b541784701FD8d65F11fc0598FBC6332,0.00,5.442048,13675970


In [33]:
df_liq_providers = df_liq_provider_events.groupby('addr').sum()
df_liq_providers.drop(columns=['block'], inplace=True)
df_liq_providers.sort_values(by=['usdn'], ascending=False, inplace=True)
df_liq_providers

,usdn,3crv
addr,,
0x094d12e5b541784701FD8d65F11fc0598FBC6332,2.756561e+08,2.862196e+09
0x35796DAc54f144DFBAD1441Ec7C32313A7c29F39,6.443519e+06,4.558611e+07
0x77eD360E62cA6f2D1582F00da8B0b8489cA1e2E2,5.056966e+06,4.675493e+07
0x7A0a6906De7714d27413f5092ED1a0636A3FBc9A,3.017516e+06,5.010975e+07
0x8C01A994639163e2724f10b6f9a71BDb94559Db9,2.231543e+06,1.660843e+06
...,...,...
0x5DB5286CEC9a28D76feb4eE22E0722e6a6912B22,0.000000e+00,3.577503e+04
0x5DB0d56FF51dABe3159741b88556db4b7020DC2a,0.000000e+00,3.072424e+06
0x5D58518c1902ED78C48D77EBe71fEEdE67419438,0.000000e+00,3.172169e+02
